In [1]:
import os
import cv2
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PyQt5 import QtCore
from ultralytics import YOLO
from urllib.request import urlopen
from keras.models import load_model
from PyQt5.QtCore import Qt, QTimer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from object_detection.utils import label_map_util
from tensorflow.keras.callbacks import TensorBoard
from keras.preprocessing import image as keras_image
from PyQt5.QtGui import QIcon, QPixmap, QImage, QTextCursor
from tensorflow.keras.metrics import categorical_crossentropy
from object_detection.utils import visualization_utils as viz_utils
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from PyQt5.QtWidgets import QApplication, QMainWindow, QFrame, QPushButton, QLabel, QFileDialog, QInputDialog, QLineEdit, QVBoxLayout, QWidget, QDialog, QScrollArea, QSizePolicy, QTextEdit, QMessageBox

In [2]:
class LicensePlateEntryDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle('Enter License Plate')
        self.setGeometry(200, 200, 300, 100)

        self.license_plate_entry = QLineEdit(self)
        self.license_plate_entry.setPlaceholderText('Enter license plate')
        self.license_plate_entry.setStyleSheet("background-color: white;")

        add_plate_button = QPushButton('Add License Plate', self)
        add_plate_button.clicked.connect(self.accept)

        layout = QVBoxLayout()
        layout.addWidget(self.license_plate_entry)
        layout.addWidget(add_plate_button)
        self.setLayout(layout)

    def get_license_plate(self):
        return self.license_plate_entry.text()

In [3]:
class MainWindow(QMainWindow):

    ######################################################################### Settings #################################################################################

    def __init__(self, app):
        super().__init__()

        # Set Up The Main Window
        self.setWindowTitle("LCPD")

        # Set Background Color
        self.setStyleSheet("background-color: #6096b5;")

        # Get The Screen Resolution
        screen_resolution = app.desktop().screenGeometry()
        width, height = screen_resolution.width(), screen_resolution.height()
        self.setGeometry(0, 0, width, height)

        # Set Window Icon
        icon_path = r"D:\University\Project\Media\Icon\Icon.png"
        self.setWindowIcon(QIcon(icon_path))

        # Box
        self.box = QFrame(self)
        self.box.setGeometry(height - height, height - 130, width, width)
        self.box.setStyleSheet("background-color: #9cc2d9; border: 5px solid black;")

        # Fullscreen Button
        self.fullscreen_button = QPushButton('Fullscreen', self)
        self.fullscreen_button.setGeometry(width - 230, height - 115, 100, 70)
        self.fullscreen_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.fullscreen_button.clicked.connect(self.fullscreen)

        # Close Button
        self.close_button = QPushButton('Close', self)
        self.close_button.setGeometry(width - 120, height - 115, 100, 30)
        self.close_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.close_button.clicked.connect(self.close_application)

        # Clear Button
        self.clear_button = QPushButton('Clear', self)
        self.clear_button.setGeometry(width - 120, height - 75, 100, 30)
        self.clear_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.clear_button.clicked.connect(self.clear)

        # Load Picture Button
        self.load_picture_button = QPushButton('Load Picture', self)
        self.load_picture_button.setGeometry(width - 1910, height - 115, 100, 30)
        self.load_picture_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.load_picture_button.clicked.connect(self.load_picture)

        # Load Video Button
        self.load_video_button = QPushButton('Load Video', self)
        self.load_video_button.setGeometry(width - 1910, height - 75, 100, 30)
        self.load_video_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.load_video_button.clicked.connect(self.load_video)

        # Detect Cars In Picture Button
        self.detect_cars_in_picture_button = QPushButton('Detect Cars In Picture', self)
        self.detect_cars_in_picture_button.setGeometry(width - 1800, height - 115, 150, 30)
        self.detect_cars_in_picture_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.detect_cars_in_picture_button.clicked.connect(self.detect_cars_in_picture)

        # Detect Cars In Video Button
        self.detect_cars_in_video_button = QPushButton('Detect Cars In Video', self)
        self.detect_cars_in_video_button.setGeometry(width - 1800, height - 75, 150, 30)
        self.detect_cars_in_video_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.detect_cars_in_video_button.clicked.connect(self.detect_cars_in_video)
        
        # Real-time Detection Button
        self.realtime_button = QPushButton('Real-time Car Detection', self)
        self.realtime_button.setGeometry(width - 1910, height - 35, 260, 30)
        self.realtime_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.realtime_button.clicked.connect(self.realtime_detection)
        
        # Detect Plate Area Button
        self.detect_plate_area_in_picture_button = QPushButton('Detect Plate Area In Picture', self)
        self.detect_plate_area_in_picture_button.setGeometry(width - 1640, height - 115, 200, 30)
        self.detect_plate_area_in_picture_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.detect_plate_area_in_picture_button.clicked.connect(self.detect_plate_area_in_picture)
        
        # Add License Plate Button
        self.add_plate_button = QPushButton('Add Plate Number', self)
        self.add_plate_button.setGeometry(width - 1430, height - 115, 165, 30)
        self.add_plate_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.add_plate_button.clicked.connect(self.show_license_plate_entry_dialog)
        
        # Show Plate List Button
        self.show_plate_list_button = QPushButton('Show Plate Number List', self)
        self.show_plate_list_button.setGeometry(width - 1430, height - 75, 165, 30)
        self.show_plate_list_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.show_plate_list_button.clicked.connect(self.show_plate_list)
        
        # Detect Plate Numbers Button
        self.detect_plate_numbers_button = QPushButton('Detect Plate Numbers', self)
        self.detect_plate_numbers_button.setGeometry(width - 1640, height - 35, 200, 30)
        self.detect_plate_numbers_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.detect_plate_numbers_button.clicked.connect(self.detect_plate_numbers)
        
        # Load Plate Picture Button
        self.load_plate_picture_button = QPushButton('Load Plate Picture', self)
        self.load_plate_picture_button.setGeometry(width - 1640, height - 75, 200, 30)
        self.load_plate_picture_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.load_plate_picture_button.clicked.connect(self.load_plate_picture)
        
        # Detect Lane Line In The Picture Button
        self.detect_line_in_picture_button = QPushButton('Detect Lane Line In Picture', self)
        self.detect_line_in_picture_button.setGeometry(width - 1255, height - 115, 260, 30)
        self.detect_line_in_picture_button.setStyleSheet("background-color: #f2ec35; color: black; font-weight: bold;")
        self.detect_line_in_picture_button.clicked.connect(self.detect_line_in_picture)
        
        # List to store license plates
        self.plate_list = []

        # QLabel To Show The Loaded Picture Or Video
        self.picture_and_video_label = QLabel(self)
        self.picture_and_video_label.setGeometry(width - width, height - height, width - 960, height - 130)
        self.picture_and_video_label.setAlignment(Qt.AlignCenter)

        # QLabel To Show The Result Picture Or Video
        self.result_label = QLabel(self)
        self.result_label.setGeometry(width - 960, height - height, width - 960, height - 130)
        self.result_label.setAlignment(Qt.AlignCenter)

        # QLabel to QTextEdit for the license plate list
        self.plate_list_label = QTextEdit(self)
        self.plate_list_label.setGeometry(width - width, height - height, width, height - 130)
        self.plate_list_label.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        self.plate_list_label.setStyleSheet("background-color: white; border: 5px solid black; padding: 10px; font-weight: bold;")
        self.plate_list_label.hide() 
        
        # YOLO Model
        self.model = YOLO(r"D:\University\Project\Models\yolo\yolov8m.pt")
        
        # Trained Model For Plate Area
        PBTXT = r"D:\University\Project\Models\models_for_project\label_map.pbtxt"
        CONFIG = r"D:\University\Project\Models\models_for_project\pipeline.config"
        MODEL = r"D:\University\Project\Models\models_for_project\exported-models-V2\my_model\saved_model"
        
        # Trained Number Detection Model 
        self.model_number = load_model(r"D:\University\Project\Models\trained numebr detection\LeNet_model.h5")
        
        # Trained Alphabet Detection Model 
        self.model_alphabet = load_model(r"D:\University\Project\Models\trained persian alphabet detection\trained_model.h5")
        
        # Label Mapping For Alphabet Classes
        self.label_mapping_alphabet = {0: "0", 1: "1", 2: "2", 3: "3", 4: "4", 5: "5", 6: "6", 7: "7", 8: "8", 9: "9", 10: "a", 11: "b", 12: "p", 13: "t",14: "s", 15: "jim",
                                       16: "ch", 17: "ha", 18: "kh", 19: "d", 20: "zal", 21: "r", 22: "z", 23: "zh", 24: "sin", 25: "shin", 26: "sad", 27: "zad", 28: "ta",
                                       29: "za", 30: "e", 31: "q", 32: "f", 33: "q", 34: "k", 35: "g", 36: "l", 37: "m", 38: "n", 39: "v", 40: "h", 41: "y"}
        
        # Timer for video playback
        self.timer = QTimer(self)

    ######################################################################### Functions #################################################################################

    def fullscreen(self):
        if self.isFullScreen():
            self.showNormal()
        else:
            self.showFullScreen()

    def close_application(self):
        self.close()

    def clear(self):
        self.picture_and_video_label.clear()
        self.result_label.clear()
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()

    def load_picture(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_names, _ = QFileDialog.getOpenFileNames(self, "Select Image Files", "", "Images (*.png *.jpg *.bmp *.jpeg);;All Files (*)", options=options)

        if file_names:
            file_name = file_names[0]
            original_image = cv2.imread(file_name)
            if original_image is not None:
                self.display_image(original_image)
                self.uploaded_image = original_image

    def display_image(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, channel = image.shape
        border_size = 5
        bordered_image = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        q_img = QImage(bordered_image.data, width + 2 * border_size, height + 2 * border_size, bordered_image.strides[0], QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_img)
        self.picture_and_video_label.setPixmap(pixmap)
        self.picture_and_video_label.setScaledContents(True)

    def load_video(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Video File", "", "Videos (*.mp4 *.avi *.mkv *.webm);;All Files (*)", options=options)

        if file_name:
            self.cap = cv2.VideoCapture(file_name)
            if self.cap.isOpened():
                self.play_video()

    def play_video(self):
        self.timer.timeout.connect(self.display_video_frame)
        self.timer.start(30)

    def display_video_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            self.timer.stop()
            return
        self.display_image(frame)

    def detect_cars_in_picture(self):
        if hasattr(self, 'uploaded_image'):
            car_count = 0
            results = self.model.predict(self.uploaded_image, verbose=False)
            results = results[0]

            for i, box in enumerate(results.boxes):
                class_id = results.names[box.cls[0].item()]

                if class_id == "car":
                    cords = box.xyxy[0].tolist()
                    cords = [round(x) for x in cords]
                    conf = round(box.conf[0].item(), 2)
                    x1, y1, x2, y2 = cords

                    if conf >= 0.25:
                        car_count += 1
                        cv2.rectangle(self.uploaded_image, (x1, y1), (x2, y2), (255, 255, 0), 2)

            cv2.rectangle(self.uploaded_image, (10, 10), (325, 50), (0, 0, 0), -1)
            cv2.putText(self.uploaded_image, f"{car_count} car(s) Detected", (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 255, 51), 2)
        self.display_result_image(self.uploaded_image)
        
    def detect_cars_in_video(self):
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.timer.timeout.connect(self.detect_cars_and_display)
            self.timer.start(30)

    def detect_cars_and_display(self):
        ret, frame = self.cap.read()
        if not ret:
            self.timer.stop()
            self.cap.release()
            return

        car_count = 0
        results = self.model.predict(frame, verbose=False)
        results = results[0]

        for i, box in enumerate(results.boxes):
            class_id = results.names[box.cls[0].item()]

            if class_id == "car":
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 2)
                x1, y1, x2, y2 = cords

                if conf >= 0.25:
                    car_count += 1
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
                    
        cv2.rectangle(frame, (10, 10), (325, 50), (0, 0, 0), -1)
        cv2.putText(frame, f"{car_count} car(s) Detected", (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 255, 51), 2)
        self.display_result_image(frame)
        
    def realtime_detection(self):
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        
        webcam_url, ok = QInputDialog.getText(self, 'Webcam URL', 'Enter the webcam URL:')
        if not ok:
            return
        
        self.cap = cv2.VideoCapture(str(webcam_url))

        if not self.cap.isOpened():
            QMessageBox.warning(self, 'Error', 'Unable to open the webcam stream.')
            return

        self.timer.timeout.connect(self.detect_cars_and_display)
        self.timer.start(30)

    def display_result_image(self, result_image):
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        height, width, channel = result_image.shape
        border_size = 5
        bordered_image = cv2.copyMakeBorder(result_image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        q_img = QImage(bordered_image.data, width + 2 * border_size, height + 2 * border_size, bordered_image.strides[0], QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_img)
        self.result_label.setPixmap(pixmap)
        self.result_label.setScaledContents(True)
        
        
    def detect_plate_area_in_picture(self):
        if hasattr(self, 'uploaded_image'):
            image = cv2.cvtColor(self.uploaded_image, cv2.COLOR_BGR2RGB)
            img_np = np.array(image)
            MODEL = r"D:\University\Project\Models\models_for_project\exported-models-V2\my_model\saved_model"
            PBTXT = r"D:\University\Project\Models\models_for_project\label_map.pbtxt"
            detector = tf.saved_model.load(MODEL)
            category_index = label_map_util.create_category_index_from_labelmap(PBTXT, use_display_name=True)
            detections = detector(np.expand_dims(img_np, 0))
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
            detections['num_detections'] = num_detections
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
            height, width, channel = np.shape(image)
            image_np_with_detections = img_np.copy()
            high_prob_indices = np.where(detections['detection_scores'] > 0.10)[0]
            high_prob_boxes = detections['detection_boxes'][high_prob_indices]
            high_prob_classes = detections['detection_classes'][high_prob_indices]
            high_prob_scores = detections['detection_scores'][high_prob_indices]

            if len(high_prob_indices) > 0:
                viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    high_prob_boxes,
                    high_prob_classes,
                    high_prob_scores,
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=1,
                    min_score_thresh=0.10,
                    agnostic_mode=False,
                    skip_scores=False,
                    skip_labels=False
                )
                
            result_image = cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)
            self.display_result_image(result_image)
            
    def show_license_plate_entry_dialog(self):
        dialog = LicensePlateEntryDialog(self)

        if dialog.exec_():
            plate_number = dialog.get_license_plate()

            if plate_number:
                if plate_number in self.plate_list:
                    QMessageBox.warning(self, 'Plate Already Exists', 'The Entered Plate Number Already Exists.')
                else:
                    self.plate_list.append(plate_number)
                    self.update_plate_list_label()

    def update_plate_list_label(self):
        plate_list_str = '\n'.join(self.plate_list)
        self.plate_list_label.setText(plate_list_str)

    def show_plate_list(self):
        self.plate_list_label.setVisible(not self.plate_list_label.isVisible())
        
        
    def detect_plate_numbers(self):
        img = cv2.imread(r"D:\University\Project\Live Proccessing\preprocessed plate picture\part_4.jpg")
        img = cv2.resize(img, (60, 60))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = keras_image.img_to_array(img)
        img_array = img_array / 255.0 
        img_array = np.expand_dims(img_array, axis=0)

        predictions_alphabet = self.model_alphabet.predict(img_array)
        predicted_class_alphabet = np.argmax(predictions_alphabet)
        predicted_label_alphabet = self.label_mapping_alphabet.get(predicted_class_alphabet, "Unknown")

        def preprocess_image_number(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))
            img = 255 - img.astype(float)
            return np.expand_dims(img, axis=0)

        def predict_image_number(img):
            pred = self.model_number.predict(img)
            return pred.argmax(axis=1)[0]

        folder_path = r"D:\University\Project\Live Proccessing\preprocessed plate picture\cropped numbers"

        results = []

        for filename in sorted(os.listdir(folder_path)):
            file_path = os.path.join(folder_path, filename)

            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                processed_img_number = preprocess_image_number(file_path)
                prediction_number = predict_image_number(processed_img_number)
                results.append(str(prediction_number))

        result_str_numbers = ''.join(results)
        result_str_combined = f"{result_str_numbers[:2]}{predicted_label_alphabet}{result_str_numbers[2:]}"
        if result_str_combined in self.plate_list:
            QMessageBox.warning(self, 'Plate Already Exists', 'Already A Member, Allowed Access.')
        else:
            self.plate_list.append(result_str_combined)
            self.update_plate_list_label()

    def update_plate_list_label(self):
        plate_list_str = '\n'.join(self.plate_list)
        self.plate_list_label.setText(plate_list_str)
        self.plate_list_label.moveCursor(QTextCursor.End)
        
        
    def load_plate_picture(self):
        image_path = QFileDialog.getOpenFileName(self, "Select Plate Image", "", "Images (*.png *.jpg *.bmp *.jpeg);;All Files (*)")[0]
        if not image_path:
            return None
        image = cv2.imread(image_path)
        image = cv2.imread(image_path)
        up_crop = int(image.shape[0] * 0.1)
        down_crop = int(image.shape[0] * 0.02)
        left_crop = int(image.shape[1] * 0.10)
        right_crop = int(image.shape[1] * 0)
        image = image[up_crop:image.shape[0] - down_crop, left_crop:image.shape[1] - right_crop]

        output_folder = r"D:\University\Project\Live Proccessing\\preprocessed plate picture"
        os.makedirs(output_folder, exist_ok=True)
        
        up_crop = int(image.shape[0] * 0)
        down_crop = int(image.shape[0] * 0.05)
        left_crop = int(image.shape[1] * 0)
        right_crop = int(image.shape[1] * 0.77)
        cropped_image_1 = image[up_crop:image.shape[0] - down_crop, left_crop:image.shape[1] - right_crop]
        output_path_1 = os.path.join(output_folder, 'part_1.jpg')
        cv2.imwrite(output_path_1, cropped_image_1)
        
        up_crop = int(image.shape[0] * 0)
        down_crop = int(image.shape[0] * 0.05)
        left_crop = int(image.shape[1] * 0.43)
        right_crop = int(image.shape[1] * 0.25)
        cropped_image_2 = image[up_crop:image.shape[0] - down_crop, left_crop:image.shape[1] - right_crop]
        output_path_2 = os.path.join(output_folder, 'part_2.jpg')
        cv2.imwrite(output_path_2, cropped_image_2)

        up_crop = int(image.shape[0] * 0.2)
        down_crop = int(image.shape[0] * 0)
        left_crop = int(image.shape[1] * 0.8)
        right_crop = int(image.shape[1] * 0.03)
        cropped_image_3 = image[up_crop:image.shape[0] - down_crop, left_crop:image.shape[1] - right_crop]
        output_path_3 = os.path.join(output_folder, 'part_3.jpg')
        cv2.imwrite(output_path_3, cropped_image_3)
        
        up_crop = int(image.shape[0] * 0)
        down_crop = int(image.shape[0] * 0)
        left_crop = int(image.shape[1] * 0.20)
        right_crop = int(image.shape[1] * 0.57)
        cropped_image_4 = image[up_crop:image.shape[0] - down_crop, left_crop:image.shape[1] - right_crop]
        output_path_4 = os.path.join(output_folder, 'part_4.jpg')
        cv2.imwrite(output_path_4, cropped_image_4)

        image_path = r"D:\University\Project\Live Proccessing\preprocessed plate picture\part_1.jpg"
        image = cv2.imread(image_path)
        output_folder = r"D:\University\Project\Live Proccessing\preprocessed plate picture\cropped numbers"
        height, width = image.shape[:2]
        left_crop = int(width * 0.55)
        cropped_image_1 = image[:, :left_crop]
        cropped_image_2 = image[:, left_crop:]
        output_path_1 = os.path.join(output_folder, 'part_1_1.jpg')
        output_path_2 = os.path.join(output_folder, 'part_1_2.jpg')
        cv2.imwrite(output_path_1, cropped_image_1)
        cv2.imwrite(output_path_2, cropped_image_2)

        image_path = r"D:\University\Project\Live Proccessing\preprocessed plate picture\part_2.jpg"
        image = cv2.imread(image_path)
        height, width = image.shape[:2]
        crop_1 = int(width * 1 / 3)
        crop_2 = int(width * 2 / 3)
        cropped_image_1 = image[:, :crop_1]
        cropped_image_2 = image[:, crop_1:crop_2]
        cropped_image_3 = image[:, crop_2:]
        output_folder = r"D:\University\Project\Live Proccessing\preprocessed plate picture\cropped numbers"
        os.makedirs(output_folder, exist_ok=True)
        output_path_1 = os.path.join(output_folder, 'part_2_1.jpg')
        output_path_2 = os.path.join(output_folder, 'part_2_2.jpg')
        output_path_3 = os.path.join(output_folder, 'part_2_3.jpg')
        cv2.imwrite(output_path_1, cropped_image_1)
        cv2.imwrite(output_path_2, cropped_image_2)
        cv2.imwrite(output_path_3, cropped_image_3)

        image_path = r"D:\University\Project\Live Proccessing\preprocessed plate picture\part_3.jpg"
        image = cv2.imread(image_path)
        height, width = image.shape[:2]
        left_crop = int(width * 0.5)
        cropped_image_1 = image[:, :left_crop]
        cropped_image_2 = image[:, left_crop:]
        output_folder = r"D:\University\Project\Live Proccessing\preprocessed plate picture\cropped numbers"
        os.makedirs(output_folder, exist_ok=True)
        output_path_1 = os.path.join(output_folder, 'part_3_1.jpg')
        output_path_2 = os.path.join(output_folder, 'part_3_2.jpg')

        cv2.imwrite(output_path_1, cropped_image_1)
        cv2.imwrite(output_path_2, cropped_image_2)
        
        
    def detect_line_in_picture(self):
        if hasattr(self, 'uploaded_image'):
            result_image = self.lane_finding_pipeline(self.uploaded_image)
            self.display_result_image(result_image)
        else:
            print("Please load an image first.")

    def grayscale(self, img):
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    def canny(self, img, low_threshold, high_threshold):
        return cv2.Canny(img, low_threshold, high_threshold)

    def gaussian_blur(self, img, kernel_size):
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    def region_of_interest(self, img, vertices):
        mask = np.zeros_like(img)

        if len(img.shape) > 2:
            channel_count = img.shape[2]
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255

        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

    def draw_lines(self, img, lines, color=[255, 0, 0], thickness=10):
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    def slope_lines(self, image, lines):
        img = image.copy()
        poly_vertices = []
        order = [0, 1, 3, 2]

        left_lines = []
        right_lines = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x1 == x2:
                    pass
                else:
                    m = (y2 - y1) / (x2 - x1)
                    c = y1 - m * x1

                    if m < 0:
                        left_lines.append((m, c))
                    elif m >= 0:
                        right_lines.append((m, c))

        if left_lines and right_lines:
            left_line = np.mean(left_lines, axis=0)
            right_line = np.mean(right_lines, axis=0)

            for slope, intercept in [left_line, right_line]:
                rows, cols = image.shape[:2]
                y1 = int(rows)
                y2 = int(rows * 0.6)
                x1 = int((y1 - intercept) / slope)
                x2 = int((y2 - intercept) / slope)
                poly_vertices.append((x1, y1))
                poly_vertices.append((x2, y2))
                self.draw_lines(img, np.array([[[x1, y1, x2, y2]]]))

            poly_vertices = [poly_vertices[i] for i in order]
            cv2.fillPoly(img, pts=np.array([poly_vertices], 'int32'), color=(0, 255, 0))
            return cv2.addWeighted(image, 0.7, img, 0.4, 0.)
        else:
            return image

    def hough_lines(self, img, rho, theta, threshold, min_line_len, max_line_gap):
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                                maxLineGap=max_line_gap)
        if lines is not None:
            line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
            line_img = self.slope_lines(line_img, lines)
            return line_img
        else:
            print("No lines detected.")
            return img

    def weighted_img(self, img, initial_img, α=0.1, β=1., γ=0.):
        try:
            lines_edges = cv2.addWeighted(initial_img, α, img, β, γ)
            return lines_edges
        except Exception as e:
            print(f"Error in weighted_img: {e}")
            return initial_img


    def get_vertices(self, image):
        rows, cols = image.shape[:2]
        bottom_left = [cols * 0.15, rows]
        top_left = [cols * 0.45, rows * 0.6]
        bottom_right = [cols * 0.95, rows]
        top_right = [cols * 0.55, rows * 0.6]

        ver = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
        return ver

    def lane_finding_pipeline(self, image):
        gray_img = self.grayscale(image)
        smoothed_img = self.gaussian_blur(img=gray_img, kernel_size=5)
        canny_img = self.canny(img=smoothed_img, low_threshold=180, high_threshold=240)
        masked_img = self.region_of_interest(img=canny_img, vertices=self.get_vertices(image))
        houghed_lines = self.hough_lines(img=masked_img, rho=1, theta=np.pi / 180, threshold=20,
                                         min_line_len=20, max_line_gap=180)
        output = self.weighted_img(img=houghed_lines, initial_img=image, α=0.8, β=1., γ=0.)
        return output
    

def main():
    app = QApplication(sys.argv)
    window = MainWindow(app)
    window.show()
    sys.exit(app.exec_())

In [4]:
if __name__ == '__main__':
    main()

SystemExit: 0

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
